In [ ]:
!pip install pyomo==6.7.1

In [1]:
from pathlib import Path

import pandas as pd
import datetime
import pyomo.environ as pe
import pyomo.gdp as pyogdp

from itertools import product
import numpy as np
import plotly.express as px

/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/3463496630.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
####### EVALUATION #######
# TODO: check the values for commute and downtime counts to define the correct weight (currently it's 10)
# TODO: clean up preprocessing etc.
# TODO: plot optimised schedule with commute times
# TODO: run code for all days
# TODO: compare metric for optimised vs given schedule

####### QUESTION 2 #######
# TODO: filter for caregivers available that day
# TODO: change commute df to have bike time instead of driving time for no license
# TODO: filter / constrain for staff being able to perform task according to skills

####### QUESTION 3 #######
# TODO: create df sessions with additional for new personas -> do it for 3 different scenarios of combinations of personas or something
# TODO: run optimisation

####### "BONUS" #######
# TODO: add carbon emissions to objective function

In [2]:
excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [3]:
schedule = pd.read_excel(excel_file, sheet_name=0)
clients = pd.read_excel(excel_file, sheet_name=1)
caregivers = pd.read_excel(excel_file, sheet_name=2)

In [4]:
schedule.Prestation.unique()

array(['REPAS', 'TOILETTE', 'VIE SOCIALE', 'AIDE MENAGERE',
       'ADMINISTRATION', 'ACCOMPAGNEMENTS COURSES PA', 'FEMME DE MENAGE',
       'VISITE MEDICALE', 'FORMATION', "GARDE D'ENFANTS", 'COORDINATION',
       'HOMMES TOUTES MAINS'], dtype=object)

In [5]:
discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

In [6]:
sched_one_day = schedule.loc[schedule.Date == "2024-01-03", :]

In [7]:
sched_one_day.Prestation.value_counts()

Prestation
TOILETTE                      41
REPAS                         25
AIDE MENAGERE                 14
ACCOMPAGNEMENTS COURSES PA     4
ADMINISTRATION                 3
VIE SOCIALE                    3
Name: count, dtype: int64

In [8]:
sched_one_day = sched_one_day[~sched_one_day.Prestation.isin(discard_list)]

In [9]:
sched_one_day["idx"] = sched_one_day.index

In [10]:
from datetime import datetime

common_date = sched_one_day["Date"].iloc[0]
sched_one_day["Heure de fin"] = sched_one_day["Heure de fin"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Heure de début"] = sched_one_day["Heure de début"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Duration"] = (
    sched_one_day["Heure de fin"] - sched_one_day["Heure de début"]
)
sched_one_day["Duration"] = sched_one_day["Duration"].apply(
    lambda x: x.seconds // 60
)

In [11]:
sched_one_day

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation,idx,Duration
177,522099746,162858075,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:30:00,TOILETTE,177,60
178,87852633,776399834,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:15:00,TOILETTE,178,45
179,559277088,842621873,2024-01-03,2024-01-03 07:45:00,2024-01-03 08:15:00,TOILETTE,179,30
180,315244686,810259688,2024-01-03,2024-01-03 08:00:00,2024-01-03 08:45:00,TOILETTE,180,45
181,714782168,710283561,2024-01-03,2024-01-03 08:00:00,2024-01-03 09:00:00,TOILETTE,181,60
...,...,...,...,...,...,...,...,...
262,794098228,78005437,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,262,60
263,559475456,838320706,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:40:00,TOILETTE,263,45
264,714782168,810259688,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,264,60
265,803656603,838320706,2024-01-03,2024-01-03 19:50:00,2024-01-03 20:20:00,TOILETTE,265,30


In [12]:
before_first_client = caregivers[["ID Intervenant"]].copy()
before_first_client["ID Client"] = caregivers["ID Intervenant"]
before_first_client["Duration"] = 0 
before_first_client["Date"] = pd.to_datetime("2024-01-03")
before_first_client["Heure de début"] = before_first_client["Date"] + pd.Timedelta(hours=5)
before_first_client["Heure de fin"] = before_first_client["Date"] + pd.Timedelta(hours=5)
before_first_client["Prestation"] = "COMMUTE"
before_first_client["idx"] = before_first_client.index

In [13]:
after_last_client = caregivers[["ID Intervenant"]].copy()
after_last_client["ID Client"] = caregivers["ID Intervenant"]
after_last_client["Duration"] = 0 
after_last_client["Date"] = pd.to_datetime("2024-01-03")
after_last_client["Heure de début"] = after_last_client["Date"] + pd.Timedelta(hours=22)
after_last_client["Heure de fin"] = after_last_client["Date"] + pd.Timedelta(hours=22)
after_last_client["Prestation"] = "COMMUTE"
after_last_client["idx"] = before_first_client.index + 10000

In [14]:
sched_one_day = pd.concat([sched_one_day, before_first_client], ignore_index=True)
sched_one_day = pd.concat([sched_one_day, after_last_client], ignore_index=True)

In [15]:
sched_one_day = sched_one_day.sort_values("Heure de début")
sched_one_day.to_csv("../data/schedule_2024-03-01.csv", index=False) 

In [23]:
clients_commute = pd.read_csv("../data/commute_driving_clients.csv")

In [24]:
schedule["ID Client"].nunique()

118

In [25]:
from itertools import product, combinations

len(
    list(
        product(schedule["ID Client"].unique(), schedule["ID Client"].unique())
    )
)

13924

In [26]:
len(
    list(
        product(
            schedule["ID Client"].unique(), schedule["ID Intervenant"].unique()
        )
    )
)

2832

In [27]:
len(list(combinations(schedule["ID Client"].unique(), 2)))

6903

In [28]:
clients_commute

,clients,commute_seconds,commute_meters,source,destination,commute_method
0,"(559475456, 559475456)",0,0,559475456,559475456,driving
1,"(559475456, 559277088)",0,0,559475456,559277088,driving
2,"(559475456, 87852633)",1070,13809,559475456,87852633,driving
3,"(559475456, 243033408)",469,5880,559475456,243033408,driving
4,"(559475456, 814940942)",326,2328,559475456,814940942,driving
...,...,...,...,...,...,...
13919,"(1453084257, 334442311)",397,4232,1453084257,334442311,driving
13920,"(1453084257, 621952854)",611,9328,1453084257,621952854,driving
13921,"(1453084257, 1452858131)",952,13515,1453084257,1452858131,driving
13922,"(1453084257, 846949801)",1015,12417,1453084257,846949801,driving


In [29]:
(clients_commute.source == clients_commute.destination).sum()

118

In [30]:
list(
    clients_commute[["source", "destination", "commute_seconds"]].itertuples(
        index=False, name=None
    )
)

[(559475456, 559475456, 0),
 (559475456, 559277088, 0),
 (559475456, 87852633, 1070),
 (559475456, 243033408, 469),
 (559475456, 814940942, 326),
 (559475456, 864411156, 1511),
 (559475456, 817127257, 162),
 (559475456, 480337694, 459),
 (559475456, 861502972, 1511),
 (559475456, 87274454, 897),
 (559475456, 315244686, 248),
 (559475456, 541258495, 1280),
 (559475456, 522099746, 207),
 (559475456, 192451998, 468),
 (559475456, 552306887, 219),
 (559475456, 794098228, 1321),
 (559475456, 714782168, 409),
 (559475456, 712664807, 362),
 (559475456, 803656603, 99),
 (559475456, 446362468, 661),
 (559475456, 859852985, 1271),
 (559475456, 213156949, 493),
 (559475456, 762304005, 248),
 (559475456, 867813356, 808),
 (559475456, 658302934, 438),
 (559475456, 78899980, 75),
 (559475456, 860937824, 433),
 (559475456, 762149315, 1473),
 (559475456, 197274024, 187),
 (559475456, 466678534, 576),
 (559475456, 658187452, 482),
 (559475456, 522610892, 382),
 (559475456, 78691461, 1043),
 (559475456,

In [31]:
# Loading and merging commute data
commute_file_paths = [
    "../data/commute_driving_clients.csv",
    "../data/commute_driving_care_clients.csv",
    "../data/commute_driving_clients_care.csv",
]


commute_dataframes = [pd.read_csv(file) for file in commute_file_paths]


for df in commute_dataframes:
    if df.columns[0] not in ["pair"]:  # standardizing column names
        df.rename(columns={df.columns[0]: "pair"}, inplace=True)

commute_data_df = pd.concat(commute_dataframes, ignore_index=True)

In [32]:
commute_data_df[(commute_data_df.source == 522099746) & (commute_data_df.destination == 522099746)]

,pair,commute_seconds,commute_meters,source,destination,commute_method
1428,"(522099746, 522099746)",0,0,522099746,522099746,driving


In [33]:
caregivers_commute = pd.DataFrame({"source": schedule["ID Intervenant"].unique(), "destination": schedule["ID Intervenant"].unique()})
caregivers_commute.insert(0, "commute_meters", 0)
caregivers_commute.insert(0, "commute_seconds", 0)
caregivers_commute.insert(0, "pair", list(zip(caregivers_commute.source, df.destination)))
caregivers_commute["commute_method"] = "driving"

In [34]:
commutes = pd.concat([commute_data_df, caregivers_commute], axis=0)
commutes.to_csv("../data/commute_driving_all.csv", index=False)

In [116]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            sessions = pd.read_csv(excel_file)

            sessions["idx"] = sessions.index

            sessions["Start_time"] = (pd.to_datetime(sessions[
                "Heure de début"
            ]) - pd.to_datetime("2024-01-03 00:00:00")).dt.seconds

            sessions["Start_time"] = sessions["Start_time"].apply(
                lambda x: x // 60
            )

            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel("../data/ChallengeXHEC23022024.xlsx", sheet_name=2)
            # TODO: take kaans matrix -> filter cargeivers if they are not available today
        except FileNotFoundError:
            print("Caregiver data not found")

        try:
            df_commute = pd.read_csv(
                "../data/commute_driving_all.csv"
            )
            df_commute["commute_minutes"] = (
                df_commute["commute_seconds"] / 60
            )
            self.df_commute = df_commute
        except FileNotFoundError:
            print("Commute data not found")

        self.model = self.create_model()

    def _generate_case_durations(self):
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_start_time(self):
        return pd.Series(
            self.df_sessions["Start_time"].values,
            index=self.df_sessions["idx"],
        ).to_dict()

    def _generate_clients_commute(self):
        clients_commute = {}
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        for source, dest in product(
            self.df_sessions["ID Client"].unique(),
            self.df_sessions["ID Client"].unique(),
        ):
            if (source in cargivers) and (dest in cargivers) and (source != dest):
                continue

            clients_commute[(source, dest)] = self.df_commute.loc[
                (self.df_commute.source == source)
                & (self.df_commute.destination == dest),
                "commute_minutes",
            ].iloc[0]
        return clients_commute

    def _IDX_CLIENTS_match(self):
        return pd.Series(
            self.df_sessions["ID Client"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, caregiver) in product(cases, cases, cargivers):
            if ((self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] in cargivers) 
                & (self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0] in cargivers)):
                
                if ((self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] != caregiver) 
                    | (self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0] != caregiver)):
                    continue

            if ((self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] in cargivers) 
                & (not (self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0] in cargivers))):
                if (self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] != caregiver):
                    continue

            if ((not (self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] in cargivers))
                & (self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0] in cargivers)):
                if (self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0] != caregiver):
                    continue

            if (case1 <= case2):
                disjunctions.append((case1, case2, caregiver))

        return disjunctions
    
    def _generate_tasks(self):
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        tasks = []
        for (case, caregiver) in product(cases, cargivers):
            if (self.df_sessions.loc[self.df_sessions["idx"]==case, "ID Client"].iloc[0] in cargivers):
                if (self.df_sessions.loc[self.df_sessions["idx"]==case, "ID Client"].iloc[0] != caregiver):
                    continue

            tasks.append((case, caregiver))

        return tasks

    def _case_combinations(self):
        cases = self.df_sessions["idx"].unique()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()

        case_comb = []
        for (case1, case2) in product(cases, cases):
            if ((self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] in cargivers) 
                & (self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0] in cargivers)):
                
                if (self.df_sessions.loc[self.df_sessions["idx"]==case1, "ID Client"].iloc[0] 
                    != self.df_sessions.loc[self.df_sessions["idx"]==case2, "ID Client"].iloc[0]):
                    continue

            if case1 <= case2:
                case_comb.append((case1, case2))

        return case_comb

    def create_model(self):
        model = pe.ConcreteModel()

        # List of case IDs in home care client needs list
        model.CASES = pe.Set(initialize=self.df_sessions["idx"].tolist())
        # List of potential caregiver IDs
        model.CAREGIVERS = pe.Set(
            initialize=self.df_cargeivers["ID Intervenant"].tolist()
        )
        # Session utilisation
        model.DISJUNCTIONS = pe.Set(
            initialize=self._generate_disjunctions(), dimen=3
        )
        # List of tasks - all possible (caseID, caregiverID) combination
        model.TASKS = pe.Set(
            initialize=self._generate_tasks(), dimen=2
        )
        model.CASE_COMBINATIONS = pe.Set(
            initialize=self._case_combinations(),
            dimen=2,
        )

        # The duration (expected case time) for each operation
        model.CASE_DURATION = pe.Param(
            model.CASES, initialize=self._generate_case_durations()
        )
        # Start time of a case
        model.CASE_START_TIME = pe.Param(
            model.CASES, initialize=self._generate_start_time()
        )

        model.CLIENT_CONNECTIONS = pe.Set(
            initialize=product(
                self.df_sessions["ID Client"].unique(),
                self.df_sessions["ID Client"].unique(),
            )
        )
        model.IDX_CLIENTS = pe.Param(
            model.CASES, initialize=self._IDX_CLIENTS_match()
        )
        model.COMMUTE = pe.Param(
            model.CLIENT_CONNECTIONS,
            initialize=self._generate_clients_commute(),
        )

        # Decision Variables
        ub = 1440  # minutes in a day
        model.M = pe.Param(initialize=1e3 * ub)  # big M

        # Binary flag, 1 if case is assigned to session, 0 otherwise
        model.SESSION_ASSIGNED = pe.Var(model.DISJUNCTIONS, domain=pe.Binary)
        # commute for cargiver
        model.COMMUTE_CARE = pe.Var(
            model.DISJUNCTIONS, bounds=(0.0, 1440.0), within=pe.PositiveReals
        )
        model.DOWN_TIME_COUNTS = pe.Var(
            model.DISJUNCTIONS, within=pe.Binary
        )

        # Objective
        def objective_function(model):
            return pe.summation(model.COMMUTE_CARE) + 10 * pe.summation(
                model.DOWN_TIME_COUNTS
            )

        model.OBJECTIVE = pe.Objective(
            rule=objective_function, sense=pe.minimize
        )

        # each case can be maximum given once as source for all destinations and caregivers
        def session_assignment(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if case == case1
                    ]
                )
                <= 1
            )

        # each case can be maximum given once as destination for all sources and caregivers
        def session_assignment_2(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case2) & (case1 <= case2)
                    ]
                )
                <= 1
            )

        # each case needs to be given at least once as source or destination
        def session_assignment_3(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if case == case1
                    ]
                )
                + sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case2) & (case1 <= case2)
                    ]
                )
                >= 1
            )

        # if a case is assigned to a caregiver as source, it can't be assigned to a different caregiver as destination
        def session_assignment_4(model, case, caregiver_):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case1) & (caregiver_ == caregiver)
                    ]
                )
                + sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case2)
                        & (case1 <= case2)
                        & (caregiver_ != caregiver)
                    ]
                )
                <= 1
            )
        
        # if a case is assigned to a caregiver as destination, it also needs to be assigned as a source for this caregiver
        def session_assignment_6(model, case, caregiver_):
            return (
                (sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case1) & (caregiver_ == caregiver) | (model.IDX_CLIENTS[case1] == caregiver) & (caregiver_ == caregiver)
                    ]
                ) - sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if ((case == case2) & (caregiver_ == caregiver) | (model.IDX_CLIENTS[case2] == caregiver) & (caregiver_ == caregiver))
                        & (case1 <= case2)
                    ]
                ))
            ) == 0

        model.SESSION_ASSIGNMENT = pe.Constraint(
            model.CASES, rule=session_assignment
        )
        model.SESSION_ASSIGNMENT_2 = pe.Constraint(
            model.CASES, rule=session_assignment_2
        )
        model.SESSION_ASSIGNMENT_3 = pe.Constraint(
            model.CASES, rule=session_assignment_3
        )
        model.SESSION_ASSIGNMENT_4 = pe.Constraint(
            model.TASKS, rule=session_assignment_4
        )
        model.SESSION_ASSIGNMENT_6 = pe.Constraint(
            model.TASKS, rule=session_assignment_6
        )

        def down_time_counts(model, case1, case2, caregiver):
            return model.DOWN_TIME_COUNTS[case1, case2, caregiver] == (
                    model.SESSION_ASSIGNED[case1, case2, caregiver]
                    * int((
                        model.CASE_START_TIME[case2]
                        - (model.CASE_START_TIME[case1]
                        + model.CASE_DURATION[case1]
                        + model.COMMUTE[
                            (model.IDX_CLIENTS[case1], model.IDX_CLIENTS[case2])
                        ])
                    ) < 30)
            )

        model.DOWNTIME_CNTS = pe.Constraint(model.DISJUNCTIONS, rule=down_time_counts)

        def commute_care(model, case1, case2, caregiver):
            commute_expr = model.SESSION_ASSIGNED[case1, case2, caregiver] * (
                        model.COMMUTE[
                            (
                                model.IDX_CLIENTS[case1],
                                model.IDX_CLIENTS[case2],
                            )
                        ]
                    )
            return model.COMMUTE_CARE[case1, case2, caregiver] == commute_expr

        model.COMMUTE_CARE_CONST = pe.Constraint(
            model.DISJUNCTIONS, rule=commute_care
        )

        def no_case_overlap(model, case1, case2, caregiver):
            return [
                model.CASE_START_TIME[case1]
                + model.CASE_DURATION[case1]
                + model.COMMUTE[
                    (model.IDX_CLIENTS[case1], model.IDX_CLIENTS[case2])
                ]
                <= model.CASE_START_TIME[case2]
                + (
                    (1 - model.SESSION_ASSIGNED[case1, case2, caregiver])
                    * model.M
                ),
                model.CASE_START_TIME[case2]
                + model.CASE_DURATION[case2]
                + model.COMMUTE[
                    (model.IDX_CLIENTS[case2], model.IDX_CLIENTS[case1])
                ]
                <= model.CASE_START_TIME[case1]
                + (
                    (1 - model.SESSION_ASSIGNED[case1, case2, caregiver])
                    * model.M
                ),
            ]

        model.DISJUNCTIONS_RULE = pyogdp.Disjunction(
            model.DISJUNCTIONS, rule=no_case_overlap
        )

        pe.TransformationFactory("gdp.bigm").apply_to(model)

        return model

    def solve(self):
        # solvername = "glpk"
        solvername = "cbc"

        # solverpath_folder = (
        #     "C:\\glpk\\w64"  # does not need to be directly on c drive
        # )

        # solverpath_exe = (
        #     "C:\\glpk\\w64\\glpsol"  # does not need to be directly on c drive
        # )
        solverpath_exe = "/opt/homebrew/bin/cbc"
        solver = pe.SolverFactory(solvername, executable=solverpath_exe)

        # Add solver parameters (time limit)
        options = {"seconds": 120}
        for key, value in options.items():
           solver.options[key] = value

        # Solve model (verbose)
        solver_results = solver.solve(self.model, tee=True)
        return solver_results

In [117]:
scheduler = CareScheduler("../data/schedule_2024-03-01.csv")

In [118]:
solver_results = scheduler.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Oct 25 2023 

command line - /opt/homebrew/bin/cbc -seconds 120 -printingOptions all -import /var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/tmpuaq_9gt9.pyomo.lp -stat=1 -solve -solu /var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/tmpuaq_9gt9.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 120
Option for printingOptions changed from normal to all
Presolve 41469 (-443808) rows, 114576 (-366024) columns and 3254040 (-819504) elements
Statistics for presolved model
Original problem has 384480 integers (384480 of which binary)
Presolved problem has 114576 integers (114576 of which binary)
==== 19920 zero objective 1703 different
==== absolute objective values 1703 different
==== for integers 19920 zero objective 1703 different
==== for integers absolute objective values 1703 different
===== end objective counts


Problem has 41469 rows, 114576 columns (94656 with objective) and 3254040 elements
Column breakd

In [119]:
model = scheduler.model

In [120]:
model.OBJECTIVE.display()

OBJECTIVE : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 1088.4333348600003


In [121]:
actions = [
    k for k, v in model.SESSION_ASSIGNED.extract_values().items() if v == 1
]

In [123]:
sorted(set(list(model.SESSION_ASSIGNED.extract_values().values())))

[0.0, 1.0]

In [146]:
actions_df = pd.DataFrame(actions, columns=["idx1", "idx2", "Caregiver_ID"])

In [148]:
actions_df_1 = actions_df[["idx1", "Caregiver_ID"]]
actions_df_2 = actions_df[["idx2", "Caregiver_ID"]]
actions_df_1.columns = ["idx", "Caregiver_ID"]
actions_df_2.columns = ["idx", "Caregiver_ID"]

In [149]:
actions_df = pd.concat([actions_df_1, actions_df_2], axis=0)
actions_df = actions_df.drop_duplicates()

In [150]:
temp = scheduler.df_sessions.copy()
temp = temp.merge(actions_df, how="left", on="idx")
temp

,ID Client,Date,Heure de début,Heure de fin,Prestation,idx,Duration,Start_time,Caregiver_ID
0,818696864,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,0,0,300,818696864
1,648993440,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,1,0,300,648993440
2,739888851,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,2,0,300,739888851
3,162858075,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,3,0,300,162858075
4,1452747150,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,4,0,300,1452747150
...,...,...,...,...,...,...,...,...,...
217,609468992,2024-01-03,2024-01-03 22:00:00,2024-01-03 22:00:00,COMMUTE,130,0,1320,609468992
218,838320706,2024-01-03,2024-01-03 22:00:00,2024-01-03 22:00:00,COMMUTE,131,0,1320,838320706
219,648993440,2024-01-03,2024-01-03 22:00:00,2024-01-03 22:00:00,COMMUTE,132,0,1320,648993440
220,810259688,2024-01-03,2024-01-03 22:00:00,2024-01-03 22:00:00,COMMUTE,133,0,1320,810259688


In [153]:
temp.idx.unique()

idx
67     2
70     2
81     2
80     2
79     2
      ..
19     1
20     1
21     1
22     1
134    1
Name: count, Length: 135, dtype: int64

In [133]:
temp.Caregiver_ID.nunique()

24

In [134]:
temp.Caregiver_ID.isna().mean()

0.0

In [135]:
temp.style

,ID Client,Date,Heure de début,Heure de fin,Prestation,idx,Duration,Start_time,Caregiver_ID
0,818696864,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,0,0,300,818696864
1,648993440,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,1,0,300,648993440
2,739888851,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,2,0,300,739888851
3,162858075,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,3,0,300,162858075
4,1452747150,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,4,0,300,1452747150
5,710283561,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,5,0,300,710283561
6,842621873,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,6,0,300,842621873
7,78007874,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,7,0,300,78007874
8,480302361,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,8,0,300,480302361
9,854577575,2024-01-03,2024-01-03 05:00:00,2024-01-03 05:00:00,COMMUTE,9,0,300,854577575


In [138]:
# Loading and merging commute data
commute_file_paths = [
    "../data/commute_bicycling_clients.csv",
    "../data/commute_driving_clients.csv",
    "../data/commute_bicycling_care_clients.csv",
    "../data/commute_bicycling_clients_care.csv",
    "../data/commute_driving_care_clients.csv",
    "../data/commute_driving_clients_care.csv",
]


commute_dataframes = [pd.read_csv(file) for file in commute_file_paths]


for df in commute_dataframes:
    if df.columns[0] not in ["pair"]:  # standardizing column names
        df.rename(columns={df.columns[0]: "pair"}, inplace=True)

commute_data_df = pd.concat(commute_dataframes, ignore_index=True)

In [139]:
commute_data_df["commute_minutes"] = commute_data_df["commute_seconds"] / 60

In [140]:
caregivers["Commute Method"] = "driving"  # map commute method
jan24_df = temp.merge(
    caregivers[["ID Intervenant", "Commute Method"]],
    right_on="ID Intervenant",
    left_on="Caregiver_ID",
    how="left",
)  # merge with agenda data

jan24_df["Start DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de début"].astype(str)
)
jan24_df["End DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de fin"].astype(str)
)

commute_data_df[["source", "destination"]] = commute_data_df[
    "pair"
].str.extract(r"\((\d+), (\d+)\)")
commute_data_df.drop(columns="pair", inplace=True)
commute_data_df.set_index(
    ["source", "destination", "commute_method"], inplace=True
)

# Function to compute commute and wait times
def compute_commute_and_wait_times(df, commute_data_df):
    df["Wait Time"] = 0
    df["Commute Time"] = 0

    for intervenant_id in df["ID Intervenant"].unique():
        intervenant_data = df[df["ID Intervenant"] == intervenant_id]

        for date in intervenant_data["Date"].unique():
            daily_data = intervenant_data[
                intervenant_data["Date"] == date
            ].sort_values(by="Start DateTime")
            prev_end_time = None
            prev_client_id = (
                intervenant_id  # intervenant ID for the first trip
            )

            for index, row in daily_data.iterrows():
                source_id = str(prev_client_id)
                destination_id = str(row["ID Client"])
                commute_method = row["Commute Method"]

                try:
                    commute_time = commute_data_df.loc[
                        (source_id, destination_id, commute_method),
                        "commute_minutes",
                    ]
                except KeyError:
                    commute_time = 0
                # Calculate wait time
                if prev_end_time is not None:
                    wait_time = (
                        row["Start DateTime"] - prev_end_time
                    ).total_seconds() / 60
                else:
                    wait_time = 0

                # Update the dataframe
                df.loc[index, "Wait Time"] = wait_time
                df.loc[index, "Commute Time"] = commute_time

                # Update previous end time and client ID
                prev_end_time = row["End DateTime"]
                prev_client_id = row[
                    "ID Client"
                ]  # Update to client ID for subsequent trips

    return df

/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:12: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [141]:
def plot_agenda_for_intervenant(intervenant_id, jan24_df, commute_data_df):
    intervenant_agenda = jan24_df[jan24_df["ID Intervenant"] == intervenant_id]
    intervenant_agenda_sorted = intervenant_agenda.sort_values(
        by=["Date", "Heure de début"]
    )

    df_timeline = intervenant_agenda_sorted.copy()
    df_timeline["Start"] = pd.to_datetime(df_timeline["Start DateTime"])
    df_timeline["Finish"] = pd.to_datetime(df_timeline["End DateTime"])
    df_timeline["Task"] = df_timeline["Prestation"]
    df_timeline["Resource"] = df_timeline["ID Intervenant"].astype(str)
    df_timeline["ID Client"] = df_timeline["ID Client"].astype(str)

    # Compute wait and commute times
    df_timeline = compute_commute_and_wait_times(df_timeline, commute_data_df)

    fig = px.timeline(
        df_timeline,
        x_start="Start",
        x_end="Finish",
        y="Task",
        hover_data=["ID Client", "Wait Time", "Commute Time"],
    )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=f"Agenda for Intervenant ID: {intervenant_id}")
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(
                            count=1,
                            label="1D",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(
                            count=7,
                            label="1W",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(step="all"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        )
    )

    fig.show()

    return df_timeline

In [142]:
for intervenant_id in jan24_df["ID Intervenant"].unique():
    plot_agenda_for_intervenant(intervenant_id, jan24_df, commute_data_df)

/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.433333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.9833333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.3833333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.583333333333332' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



...

/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.966666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.8666666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.583333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.9166666666666665' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



/var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/ipykernel_63950/1110905233.py:63: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9666666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

